In [1]:
# imports.py
from support import *

import time
from datetime import datetime
from itertools import product
from math import erf, sqrt
import sys

# start the ib loop
util.startLoop()

# connect to the market set in variables.json
try:
    if not ib.isConnected():
        ib = get_connected(market, 'live')
except Exception as e:
    ib = get_connected(market, 'live')

# from json
a = assign_var(market) + assign_var('common')
for v in a:
    exec(v)

# reset the log
with open(logpath+'test.log', 'w'):
    pass # clear the run log

util.logToFile(logpath+'test.log')
    
jup_disp_adjust() # adjust jupyter's display

In [8]:
# get pnl data (asyncio)
async def pnlcoro(ib):
    '''Gets the pnl object'''
    acct = ib.managedAccounts()[0]
    pnl = ib.reqPnL(acct)
    await ib.pnlEvent
    return pnl

# pnl object raises AssertionError if run the second time!!
try:
    pnl = ib.run(pnlcoro(ib))
except AssertionError:
    pass

# convert to dictionary and remove modelcode
pnldict = pnl.dict()
del pnldict['modelCode']

acsum = get_acc_summary(ib)

# merge the dictionaries
keydata = {**pnldict, **acsum}

In [9]:
keydata

{'account': 'U9329809',
 'dailyPnL': 97646.23421355347,
 'unrealizedPnL': -130695.89232714652,
 'realizedPnL': 25298.084858000002,
 'NLV': 24382604.79,
 'initmargin': 8687979.69,
 'unrealzPnL': -133641.8,
 'realzPnL': 27893.82,
 'avFunds': 15694625.09}

In [7]:
get_acc_summary(ib)

{'NLV': 24379764.37,
 'initmargin': 8786946.09,
 'unrealzPnL': -133886.49,
 'realzPnL': 25298.08,
 'avFunds': 15592818.28}

In [ ]:
get_df_buys(ib, market, prec)

In [ ]:
trades = ib.trades()
all_trades_df = util.df(t.contract for t in trades).join(util.df(t.orderStatus for t in trades)).join(util.df(t.order for t in trades), lsuffix='_')
all_trades_df.rename({'lastTradeDateOrContractMonth': 'expiry'}, axis='columns', inplace=True)
trades_cols = ['conId', 'symbol', 'localSymbol', 'secType', 'expiry', 'strike', 'right', 
               'orderId', 'permId', 'action', 'totalQuantity', 'lmtPrice', 'status']
trades_df = all_trades_df[trades_cols]
fills_df = trades_df[trades_df.status == 'Filled']